In [190]:
import numpy as np
import pandas as pd
import pulp
import itertools
from math import radians
import scipy as sp
from sklearn.neighbors import DistanceMetric

In [214]:
# import dataframe which contains demand locations co-ordinates
df = pd.read_csv('Demands.csv')
df

,latitude,longitude,demand
0,41.927897,-70.141155,14
1,42.306619,-71.473414,10
2,41.982628,-71.790283,16
3,42.792484,-71.404969,15
4,42.722427,-71.368172,17
5,42.752336,-71.705351,15
6,41.677468,-71.308487,15
7,41.463087,-70.631341,18
8,42.840534,-70.956215,11


In [215]:
# adding 3 potential warehouse locations coordinates
W_1 = pd.DataFrame({'latitude':[42.340335], 'longitude':[-71.131830], 'demand':[0]})
W_2 = pd.DataFrame({'latitude':[42.392390], 'longitude':[-71.139340], 'demand':[0]})
W_3 = pd.DataFrame({'latitude':[42.294840], 'longitude':[-71.072297], 'demand':[0]})

In [216]:
# new dataframe with potential warehouse locations
df1 = pd.concat([W_1, df]).reset_index(drop = True)
df2 = pd.concat([W_2, df]).reset_index(drop = True)
df3 = pd.concat([W_3, df]).reset_index(drop = True)

In [206]:
# function for creating distance matrix for each warehouse locations
def get_dist_matrix(df):

    df['latitude'] = np.radians(df['latitude'])
    df['longitude'] = np.radians(df['longitude'])
    df[['latitude','longitude']].to_numpy()
    dist = DistanceMetric.get_metric('haversine')
    distance = pd.DataFrame(dist.pairwise(df[['latitude','longitude']].to_numpy())*6373,  columns=df.index.unique(), index=df.index.unique())
    return distance

In [207]:
# function for solving LP with pulp
def get_min_dist(distance,vehicle_count,customer_count,vehicle_capacity,df):
    for vehicle_count in range(1,vehicle_count+1):

        # definition of LpProblem
        problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

        # definition of variables which are 0/1(decisiion variables if a route is travelled or not)
        x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

        # add objective function (minimizing travel distances of all the vehicles)
        problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                              for k in range(vehicle_count) 
                              for j in range(customer_count) 
                              for i in range (customer_count))

        # s.t constraints
        # constraint 1 - only 1 visit to every demand point
        for j in range(1, customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count) 
                                  for k in range(vehicle_count)) == 1 

        # constraint 2 - departing from depot(warehouse node-0)
        for k in range(vehicle_count):
            problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
            problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

        # constraint 3 - no of vehicles entering and leaving a demand point is same
        for k in range(vehicle_count):
            for j in range(customer_count):
                problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                      for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

        # constraint 4 - capacity constraint, delivery capacity must not exceed vehicle capacity
        for k in range(vehicle_count):
            problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 


        # constraint 5 - eliminating subtours()
        subtours = []
        for i in range(2,customer_count):
             subtours += itertools.combinations(range(1,customer_count), i)

        for s in subtours:
            problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1


        # print vehicle_count which needed for solving problem
        # print calculated minimum distance value
        if problem.solve() == 1:
            print('Vehicle Requirements:', vehicle_count)
            print('Moving Distance kms:', pulp.value(problem.objective))
            break
            
            return vehicle_count, pulp.value(problem.objective)

In [208]:
def get_warehouse_optimal_dist(df,vehicle_count,vehicle_capacity,customer_count=len(df)):
    dist_mat = get_dist_matrix(df)
    result = get_min_dist(dist_mat, vehicle_count, customer_count, vehicle_capacity,df)
    return result

In [217]:
get_warehouse_optimal_dist(df1,vehicle_count=4,vehicle_capacity=50)

Vehicle Requirements: 3
Moving Distance kms: 581.6600524669502
